In [1]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from threading import Thread
import numpy as np
from playsound import playsound
import imutils 
import time
import dlib
import cv2
from imutils import face_utils
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def alarm(path):
    playsound.playsound(path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    
    EAR = (A+B)/(2.0*C)
    return EAR


In [3]:
my_alarm_path = "loving-you.mp3"

In [4]:
EYE_aspect_ratio_threshold = 0.3
EYE_aspect_ratio_conse_frames = 48

COUNTER = 0
ALARM_ON = False

In [5]:
print("----Loading 68 face landmarks predictor----")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

print("\n\n")
print("\t---Successfully loaded---")

----Loading 68 face landmarks predictor----



	---Successfully loaded---


In [6]:
(left_st, left_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_st, right_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [7]:
left_st, left_end

(42, 48)

In [8]:
right_st, right_end

(36, 42)

In [9]:
print("Starting video stream thread...\n")
vs = VideoStream(src=0).start()
time.sleep(1.0)

Starting video stream thread...



In [ ]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray,0)
    
    for rect in rects:
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[left_st:left_end]
        rightEye = shape[right_st:right_end]
        
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        
        if ear < EYE_aspect_ratio_threshold:
            COUNTER += 1

            if COUNTER >= EYE_aspect_ratio_conse_frames:                
                if not ALARM_ON:
                    ALARM_ON = True                    
                    pygame.mixer.init()
                    pygame.mixer.music.load("seagulls-chatting.mp3")
                    pygame.mixer.music.play()

                cv2.putText(frame, "DROWSINESS ALERT...Don't sleep!", (10, 30),
                    cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 2)
        else:
            COUNTER = 0
            ALARM_ON = False

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break

        cv2.destroyAllWindows()
vs.stop()

In [22]:
vs.stop()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
vs.stream.stream.release()

In [16]:
shape[2:4]

array([[281, 177],
       [285, 191]])